**Import libraries**

In [1]:
import pandas as pd
from operator import itemgetter
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import requests
import plotly.graph_objects as go

**Try out Vader Sentiment Analyzer**

In [2]:
#call analyser object
analyser = SentimentIntensityAnalyzer()

In [3]:
def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

In [4]:
sentiment_analyzer_scores("Today is an okay day.")

{'neg': 0.0, 'neu': 0.678, 'pos': 0.322, 'compound': 0.2263}

In [5]:
sentiment_analyzer_scores("Today is an amazing day!")

{'neg': 0.0, 'neu': 0.494, 'pos': 0.506, 'compound': 0.6239}

In [6]:
sentiment_analyzer_scores("Today is an amazing day! Vader is working but it took a while")

{'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'compound': 0.4003}

In [7]:
sentiment_analyzer_scores("Today kind of sucked.")

{'neg': 0.523, 'neu': 0.477, 'pos': 0.0, 'compound': -0.5095}

**Pull in Live News Data from News API**

In [29]:
url = ('http://newsapi.org/v2/top-headlines?'
       #'q=' + input('company: ') + '&'
       'country=us&'
       'category=business&'
       'from=2020-03-01&'
       'sortBy=popularity&'
       'pageSize=100&'
       'apiKey=' + input('api key: '))

response = requests.get(url)
print('Total Results: ' + str(response.json().get('totalResults')))

api key: 798c7707bbe8404199296c0521a99f61
Total Results: 70


**Perform Analysis on the News Title**

In [30]:
news = response.json().get('articles')

#add title score:
for item in news:
    if item['title'] is None:
        continue
    item.update({'title_score': sentiment_analyzer_scores(item['title'])})
    if item['description'] is None:
        continue
    item.update({'desc_score': sentiment_analyzer_scores(item['description'])})

In [31]:
news_df = pd.DataFrame(news)
print('row and column: ' + str(news_df.shape))
news_df.head()

row and column: (70, 10)


,source,author,title,description,url,urlToImage,publishedAt,content,title_score,desc_score
0,"{'id': 'bloomberg', 'name': 'Bloomberg'}",None,Apple’s Cook Offers Work From Home This Week t...,None,https://www.bloomberg.com/tosv2.html?vid=&uuid...,None,2020-03-08T20:23:41Z,"To continue, please click the box below to let...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",NaN
1,"{'id': 'reuters', 'name': 'Reuters'}",Megan Davies,"Investor fears rise over recession, bear marke...","The words ""bear market"" and ""recession"" are be...",https://www.reuters.com/article/us-health-coro...,https://s3.reutersmedia.net/resources/r/?m=02&...,2020-03-08T20:16:18Z,NEW YORK (Reuters) - The words “bear market” a...,"{'neg': 0.337, 'neu': 0.663, 'pos': 0.0, 'comp...","{'neg': 0.142, 'neu': 0.743, 'pos': 0.115, 'co..."
2,"{'id': None, 'name': 'Bitcoinist.com'}",None,"Bitcoin Price Consolidating, is a Big Move Abo...","Bitcoin price continues to look bearish, yet a...",https://bitcoinist.com/bitcoin-price-consolida...,https://bitcoinist.com/wp-content/uploads/2020...,2020-03-08T19:03:03Z,"Bitcoin price continues to look bearish, yet a...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.0, 'neu': 0.782, 'pos': 0.218, 'comp..."
3,"{'id': 'fox-news', 'name': 'Fox News'}",Michael Hollan,Family accidentally orders 12 years' worth of ...,There’s nothing worse than not having enough t...,https://www.foxnews.com/lifestyle/12-years-toi...,https://static.foxnews.com/foxnews.com/content...,2020-03-08T18:53:56Z,There’s nothing worse than not having enough t...,"{'neg': 0.18, 'neu': 0.677, 'pos': 0.143, 'com...","{'neg': 0.0, 'neu': 0.801, 'pos': 0.199, 'comp..."
4,"{'id': None, 'name': 'Oregonlive.com'}",Laura Gunderson | The Oregonian/OregonLive,New coronavirus case at OHSU Hospital in Portl...,OSHU informed its employees over the weekend t...,https://www.oregonlive.com/coronavirus/2020/03...,https://www.oregonlive.com/resizer/PoxhmM45EGw...,2020-03-08T18:53:02Z,This is breaking news and will be updated.\r\n...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.0, 'neu': 0.806, 'pos': 0.194, 'comp..."


In [32]:
#other ways to add title score object to news


#add title score way 2 -----------------------:

# def add_title_scoore():
#     for item in news:
#         score = {'title_score' : sentiment_analyzer_scores(item['title'])}
#     return news
# add_title_scoore()


#add title score way 3 -----------------------:

# result = [dict(item, scoreeeeeeeeeeeeeee=sentiment_analyzer_scores(item['title'])) for item in news]
# print(result)


#this works append static object -------------:

# for item in news:
#     item.update({'scooooooooooooooooore': 'baaaaaaaaaaaaaaaaaad'})
# print(news)

**Plot Title Sentiment Score**

In [33]:
def graph_sentiment(text, score, graph_title):
    x = text
    y = [d.get('neg') for d in score]
    y1 = [d.get('neu') for d in score]
    y2 = [d.get('pos') for d in score]
    y3 = [d.get('compound') for d in score]

    fig = go.Figure(go.Bar(x=x, y=y, name='Negative', marker_color='#EE7674'))
    fig.add_trace(go.Bar(x=x, y=y1, name='Neutral', marker_color='#247BA0'))
    fig.add_trace(go.Bar(x=x, y=y2, name='Positive'))
    fig.add_trace(go.Bar(x=x, y=y3, name='Compound'))

    fig.update_layout(barmode='stack', title=graph_title)
    fig.show()


graph_sentiment(news_df['title'], news_df['title_score'], 'Title Sentiment')

In [ ]:
graph_sentiment(news_df['description'], news_df['desc_score'], 'Description Sentiment')

***How to interpret Compound Values?*** 
<p>The compound score is computed by summing the valence scores of each word in the lexicon, adjusted according to the rules, and then normalized to be between -1 (most extreme negative) and +1 (most extreme positive). This is the most useful metric if you want a single unidimensional measure of sentiment for a given sentence. Calling it a 'normalized, weighted composite score' is accurate.</p>

**Next Step: Populate the score on a daily basis**
<p> Use the mean of the headline score for each day </p>

In [ ]:
**Next Step: Correlate the score of the daily movement of S\\&P and **